# <center>Lyft Motion Prediction for Autonomous Vehicles</center>
<center> Level 5 Autonomy <center>
<img src="https://storage.googleapis.com/kaggle-media/competitions/Lyft-Kaggle/Motion%20Prediction/BP9I1484%20(1).jpg" alt="drawing" width="600"/>


## Content

**[1. Introduction](#heading--1)**

  * [1.1. Goal of the Competition](#heading--1-1)
  * [1.2. About Dataset](#heading--1-2)
  * [1.3. Prediction Dataset sample](#heading--1-3)

**[2. Exploratory Data Analysis](#heading--2)**

  * [2.1. Understanding the data](#heading--2-1)
  * [2.2. Competition Dataset format](#heading--2-2)
      * [2.2.1. Scenes](#heading--2-2-1)
      * [2.2.2. Frames](#heading--2-2-2)
      * [2.2.3. Agents](#heading--2-2-3)
      * [2.2.4. Traffic Light Faces](#heading--2-2-4)     
**[3. Basic Model](#heading--3)

----

## 1. Introduction <a name="heading--1"/>

### 1.1. Goal of the Competition <a name="heading--1-1"/>

Lyft is working on to achevie the level 5 autonomy for AV (Autonomus Vehicle). For that AV shoud have capabilty to guess the movement of vechicle around it .[Previous year competition](http://kaggle.com/c/3d-object-detection-for-autonomous-vehicles/) was to identified the 3D object around the AV. This year competition is to predict their movement around the AV. its kind of Predicting the movement of traffic agents around the AV, such as cars, cyclists, and pedestrians. So you have to built a motion prediction model for self driving vehicle. Watch the video for more clearification.
[![](http://img.youtube.com/vi/AsfB5o45F5s/0.jpg)](http://www.youtube.com/watch?v=AsfB5o45F5s "Self Driving Car")


### 1.2. About Dataset <a name="heading--1-2"/>

For this competitons Lyft collected the largest traffic agent motion data. This dataset includes the logs of movement of cars, cyclists, pedestrians, and other traffic agents encountered by autonomous fleet. These logs come from processing raw lidar, camera, and radar data through Lyft perception systems and are ideal for training motion prediction models. The dataset includes: 1000+ hours of traffic agent movement, 16K miles of data from 23 vehicles, 15K semantic map annotations. [Read More](https://self-driving.lyft.com/level5/prediction/)

### 1.3. Prediction Dataset sample <a name="heading--1-3"/>
The dataset consists of 170,000 scenes capturing the environment around the autonomous vehicle. Each scene encodes the state of the vehicle’s surroundings at a given point in time. Lets understand the dataset. The red box shown in figure in AV and all other yellow boxes are traffic agents  such as cars, cyclists, and pedestrians. The red line around the yellow box show there predicted motion and blue one are showing there real motion

<table><tr>
<td> <img src="https://self-driving.lyft.com/wp-content/uploads/2020/06/motion_dataset_lrg_redux.gif" alt="drawing" width="500"/> </td>
<td> <img src="https://self-driving.lyft.com/wp-content/uploads/2020/06/motion_dataset_2-1.png" alt="drawing" width="500"/></td>
</tr></table>




## 2. Exploratory Data Analysis <a name="heading--2"/>
### 2.1. Understanding the data <a name="heading--2-1"/>
The data packed in .zarr file format.  zarr data format is to store and read these numpy structured arrays from disk. Zarr allows us to write very large (structured) arrays to disk in n-dimensional compressed chunks 

In [ ]:
!pip install pymap3d==2.1.0
!pip install -U l5kit

In [ ]:
import zarr
import numpy as np

z = zarr.open("./dataset.zarr", mode="w", shape=(500,), dtype=np.float32, chunks=(100,))

# We can write to it by assigning to it. This gets persisted on disk.
z[0:150] = np.arange(150)
print(z.info)

# Reading from a zarr array is as easy as slicing from it like you would any numpy array. 
# The return value is an ordinary numpy array. Zarr takes care of determining which chunks to read from.
print(z[:10])
print(z[::20]) # Read every 20th value

### 2.2. Competition Dataset format <a name="heading--2-2"/>
The 2020 Lyft competition dataset is stored in four structured arrays: `scenes`, `frames`, `agents` and `tl_faces`.

Note: in the following all `_interval` fields assume that information is stored consecutively in the arrays.
This means that if `frame_index_interval` for `scene_0` is `(0, 100)`, frames from `scene_1` will start from index 100 in the frames array.

#### 2.2.1. Scenes <a name="heading--2-2-1"/>
A scene is identified by the host (i.e. which car was used to collect it) and a start and end time. 
It consists of multiple frames (=snapshots at discretized time intervals). 
The scene datatype stores references to its corresponding frames in terms of the start and end index within the frames array (described below). 
The frames in between these indices all correspond to the scene (including start index, excluding end index).

```python
SCENE_DTYPE = [
    ("frame_index_interval", np.int64, (2,)),
    ("host", "<U16"),  # Unicode string up to 16 chars
    ("start_time", np.int64),
    ("end_time", np.int64),
]
```

#### 2.2.2. Frames <a name="heading--2-2-2"/>
A frame captures all information that was observed at a time. This includes

- the timestamp, which the frame describes;
- data about the ego vehicle itself such as rotation and position;
- a reference to the other agents (vehicles, cyclists and pedestrians) that were captured by the ego's sensors;
- a reference to all traffic light faces (see below) for all visible lanes.

The properties for both agents and traffic light faces are stored in their two respective arrays. 
The frame contains only pointers to these stored objects given by a start and an end index in these arrays (again, start is included while end excluded).

```python
FRAME_DTYPE = [
    ("timestamp", np.int64),
    ("agent_index_interval", np.int64, (2,)),
    ("traffic_light_faces_index_interval", np.int64, (2,)),
    ("ego_translation", np.float64, (3,)),
    ("ego_rotation", np.float64, (3, 3)),
]
```

#### 2.2.3. Agents <a name="heading--2-2-3"/>
An agent is an observation by the AV of some other detected object. 
Each entry describes the object in terms of its attributes such as position and velocity, gives the agent a tracking number to track it over multiple frames (but only within the same scene!) and its most probable label. 
The label is described as an array of probabilities over each defined class associated with them, 
the possible labels are defined [here](https://github.com/lyft/l5kit/blob/master/l5kit/l5kit/data/zarr_dataset.py).

```python
AGENT_DTYPE = [
    ("centroid", np.float64, (2,)),
    ("extent", np.float32, (3,)),
    ("yaw", np.float32),
    ("velocity", np.float32, (2,)),
    ("track_id", np.uint64),
    ("label_probabilities", np.float32, (len(LABELS),)),
]
```

#### 2.2.4. Traffic Light Faces <a name="heading--2-2-4"/>


Our semantic map holds static information about the world only. This means it has a list of all traffic lights, but no information about how their status changes over time.
This dynamic information is instead stored in this array.
Each array's element has a unique id to link it to the semantic map, a status (if status `>0`, then the face is active - i.e., the corresponding light bulb is on, otherwise inactive / off ) and a reference to its parent traffic light.

```python
TL_FACE_DTYPE = [
    ("face_id", "<U16"),
    ("traffic_light_id", "<U16"),
    ("traffic_light_face_status", np.float32, (len(TL_FACE_LABELS,))),
]
```

In [ ]:
from l5kit.data import ChunkedDataset

dt = ChunkedDataset("../input/lyft-motion-prediction-autonomous-vehicles/scenes/train.zarr").open()
centroids= []
for idx in range(10_000):
    centroid = dt.agents[idx]["centroid"]
    centroids.append(centroid)
print(dt)
centroids[0:10]

####  WORK IN PROGRESS ...

### <p><span style="color:blue">if you like it. Feel free to upvote!!!</span>.</p>